# welcome to model benchmarking
#### this notebooks focus is on benchmarking baselines for every single model possible for tps may 2021,if you have any models to suggest comment below and i can add it into this notebook for everyone to see 😁
![testing_gif](https://cdn2.hubspot.net/hubfs/2621212/job-benchmarking-basics.gif)
### TODO for future versions
1. add and test feature engieering 
2. add more models!!!! 

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# data preprocessing 

### import data

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

### label encoding 

To keep this at baseline i am not going to preprocess the data as of now 
<br>
will add feature engineering steps in future versions of this notebook

In [ ]:
X = train_df.drop(['id', 'target'], axis=1)
le =LabelEncoder()
train_df['target'] = le.fit_transform(train_df['target'])
y = train_df['target']

# benchmarking code

A simple script to do cross validaion with StratifiedKFold using for loop

In [ ]:
results_dict = {}
def model_benchmarking(model,model_name,N_SPLITS,X,y):
    eval_list = []
    kf = StratifiedKFold(n_splits = N_SPLITS)
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):    
        # Prepare training and validation data
        X_train = X.iloc[train_idx].reset_index(drop=True)
        X_val = X.iloc[val_idx].reset_index(drop=True)
        y_train = y.iloc[train_idx].reset_index(drop=True)
        y_val = y.iloc[val_idx].reset_index(drop=True)
        trained_model = model.fit(X_train,y_train)
        pred = trained_model.predict_proba(X_val)
        evaluation = log_loss(y_val,pred)
        eval_list.append(evaluation)
    results_dict[model_name] = eval_list
    return eval_list

# model testing 

## xgboost

In [ ]:
xgb = XGBClassifier()
xgb_score = model_benchmarking(xgb,"xgboost",5,X,y)

In [ ]:
print("xgb logloss: ",np.mean(xgb_score))

## lgbm

In [ ]:
lgbm = LGBMClassifier()
lgbm_score = model_benchmarking(lgbm,"lgbm",5,X,y)

In [ ]:
print("lgbm logloss: ",np.mean(lgbm_score))

## catboost

In [ ]:
catb = CatBoostClassifier()
catb_score = model_benchmarking(catb,"catboost",5,X,y)

In [ ]:
print("catboost logloss: ",np.mean(catb_score))

## knn

In [ ]:
knn = KNeighborsClassifier()
knn_score = model_benchmarking(knn,"knn",5,X,y)

In [ ]:
print("knn logloss: ",np.mean(knn_score))

## Decision Tree

In [ ]:
decitree = DecisionTreeClassifier()
decitree_score = model_benchmarking(decitree,"Decision_Tree",5,X,y)

In [ ]:
print("Decision Tree logloss: ",np.mean(decitree_score))

## Random Forest

In [ ]:
randomforest = RandomForestClassifier()
randomforest_score = model_benchmarking(randomforest,"random_forest",5,X,y)

In [ ]:
print("random forest  logloss: ",np.mean(randomforest_score))

## Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg_score = model_benchmarking(logreg,"Logistic_Regression",5,X,y)

In [ ]:
print("Logistic Regression logloss: ",np.mean(logreg_score))

# results of benchmark test

In [ ]:
df = pd.DataFrame.from_dict(results_dict)

In [ ]:
result_df = df.mean().sort_values(ascending=True).to_frame("avg logloss")
fig = go.Figure(data=[go.Table(header=dict(values=["Model","Avg LogLoss"]),
                 cells=dict(values=[result_df.index,result_df["avg logloss"]]))
                     ])
fig.update_layout(
    title="Benchmark Results")
fig.show()

In [ ]:
# fig = px.box(df)
# fig.update_layout(
#     title="Benchmark Results",
#     xaxis_title="Models",
#     yaxis_title="Log Loss"
# )
# fig.show()

### the best model at baseline is catboost followed by lgbm  
#### to be continued with more models 😊............